# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [3]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [2]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [3]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [4]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":819393184,"wgRevisionId":819393184,"wgArticleId":473317,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All articles needing expert attention","Sociology articles needing expert attention","Media articles needing expert attention","Articles that may be too long from January 2018","All articles with un

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

In [9]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[10000:11000])

.
By having contents of communication available in form of machine readable texts, the input is analyzed for frequencies and coded into categories for building up inferences.
Reliability[edit]
Robert Weber notes: "To make valid inferences from the text, it is important that the classification procedure be reliable in the sense of being consistent: Different people should code the same text in the same way".[6] The validity, inter-coder reliability and intra-coder reliability are subject to intense methodological research efforts over long years.[4] Neuendorf suggests that when human coders are used in content analysis two coders should be used. Reliability of human coding is often measured using a statistical measure of intercoder reliability or "the amount of agreement or correspondence among two or more coders".[7]
Kinds of Text[edit]
There are five types of texts in content analysis:

written text, such as books and papers
oral text, such as speech and theatrical performance
iconic 

This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [10]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Now lets open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [11]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)



Content analysis is a research method for studying documents and communication artifacts, which can be texts of various formats, pictures, audio or video. Social scientists use content analysis to quantify patterns in communication, in a replicable and systematic manner.[1] One of the key advantage of this research method is to analyse social phenomena in a non-invasive way, in contrast to simulating social experiences or collecting survey answers.
Practices and philosophies of content analysis vary between scholarly communities. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful patterns.[2][3] After labeling a large set of media, a researcher is able to statistically estimate the proportions of patterns in the texts, as well as correlations between patterns.


We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [12]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0                                                    
1   \nContent analysis is a research method for st...
2   Practices and philosophies of content analysis...
3   Computers are increasingly used in content ana...
4                                                    
5                                                    
6   Content analysis is best understood as a broad...
7   The simplest and most objective form of conten...
8   A further step in analysis is the distinction ...
9   More generally, content analysis is research u...
10  By having contents of communication available ...
11  Robert Weber notes: "To make valid inferences ...
12  There are five types of texts in content analy...
13  Over the years, content analysis has been appl...
14  In recent times, particularly with the advent ...
15  Quantitative content analysis has enjoyed a re...
16  Recently, Arash Heydarian Pashakhanlou has arg...
17  Content analysis can als

Now we have a `DataFrame` containing all relevant text from the page ready to be
processed

If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

In [13]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<_sre.SRE_Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [14]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [15]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [13]:
#https://www.saem.org/docs/default-source/awaem/best-practices-awaem.pdf?sfvrsn=cb373dfd_0
#wikipedia_base_url = 'https://en.wikipedia.org'
#wikipedia_women_in_medicine = 'https://en.wikipedia.org/wiki/Women_in_medicine'
#women_in_medicine_save = 'wikipedia_women_in_medicine.html'
#example_text_file = 'sometextfile.txt'
#information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
#example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
#example_docx_save = 'example.docx'
saem_base_url = 'https://www.saem.org'
saem_women = 'https://www.saem.org/docs/default-source/awaem/best-practices-awaem.pdf?sfvrsn=cb373dfd_0'
saem_women_save = 'saem_women.html'

In [14]:
#wikipedia_women_in_medicine = 'https://en.wikipedia.org/wiki/Women_in_medicine'
#requests.get(wikipedia_women_in_medicine)
requests.get(saem_women)

<Response [200]>

In [15]:
SAEMRequest = requests.get(saem_women)
print(SAEMRequest.text[:100])

%����
 


In [16]:
SAEMSoup = bs4.BeautifulSoup(SAEMRequest.text, 'html.parser')
print(SAEMSoup.text[100:1000])

FE0532270E548F9B6596><45515599539D6A498362FDC3555DA03D>]/Index[82 40]/Info 81 0 R/Length 96/Prev 154733/Root 83 0 R/Size 122/Type/XRef/W[1 2 1]>>stream
Hb�eb	��	RrH��	�:�� �` !�	dh��r�����!���Hu�?���  d?
startxref
0
%%EOF
<>stream
OW2Oa��\x����C�4Ʃ
���00��������0 �7������,��LF]w;HЎ�������"�A��
  �Y�
<>stream
hެ�ms�8�?�}�L��. g:�q��q�y�8m����؊�������lpܴ��0����i�¦!�!L����#.�µ\<=a�.���t\���t1MCX�w ���� �%�aY���kB���+;Bz���	8\�	J|aے�


In [17]:
#women_in_medicine_save = 'wikipedia_women_in_medicine.html'

with open(saem_women_save, mode='w', encoding='utf-8') as f:
    f.write(SAEMRequest.text)

In [10]:
contentPTags = SAEMSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [18]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

NameError: name 'contentPTags' is not defined

In [53]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<_sre.SRE_Match object; span=(36, 37), match='2'>

In [54]:
print(regexResults.group(0))

2


In [55]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [56]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Physician', 0, 'physicians'), ('https://en.wikipedia.org/wiki/Allied_health_professions', 0, 'allied health professionals'), ('https://en.wikipedia.org/wiki/Medical_education', 0, 'medical education'), ('https://en.wikipedia.org/wiki/Specialty_(medicine)', 0, 'specialties'), ('https://en.wikipedia.org/wiki/Professionalization', 3, 'professionalizing'), ('https://en.wikipedia.org/wiki/Allied_health_professions', 3, 'allied health professions'), ('https://en.wikipedia.org/wiki/Nursing', 3, 'nursing'), ('https://en.wikipedia.org/wiki/Midwifery', 3, 'midwifery'), ('https://en.wikipedia.org/wiki/Medical_education', 3, 'medical education'), ('https://en.wikipedia.org/wiki/National_Institutes_of_Health', 8, 'National Institutes of Health')]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [13]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,,https://en.wikipedia.org/wiki/Content_analysis,0
1,\nContent analysis is a research method for st...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,3
4,,https://en.wikipedia.org/wiki/Content_analysis,4
5,,https://en.wikipedia.org/wiki/Content_analysis,5
6,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,6
7,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,7
8,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,8
9,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [57]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [58]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,NaN,"Historically and presently, in many parts of t...",NaN,None,None
1,NaN,,NaN,None,None
2,NaN,,NaN,None,None
3,NaN,"In 1540, Henry VIII of England granted the cha...",NaN,None,None
4,NaN,At the beginning of the twenty-first century i...,NaN,None,None
5,NaN,"However, the practice of medicine remains disp...",NaN,None,None
6,NaN,"Moreover, there are skews within the medical p...",NaN,None,None
7,NaN,Women continue to dominate in nursing. In 2000...,NaN,None,None
8,NaN,Biomedical research and academic medical profe...,NaN,None,None
9,NaN,The glass ceiling is used as a metaphor to con...,NaN,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [59]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Physician', 0, 'physicians'), ('https://en.wikipedia.org/wiki/Allied_health_professions', 0, 'allied health professionals'), ('https://en.wikipedia.org/wiki/Medical_education', 0, 'medical education'), ('https://en.wikipedia.org/wiki/Specialty_(medicine)', 0, 'specialties'), ('https://en.wikipedia.org/wiki/Professionalization', 3, 'professionalizing'), ('https://en.wikipedia.org/wiki/Allied_health_professions', 3, 'allied health professions'), ('https://en.wikipedia.org/wiki/Nursing', 3, 'nursing'), ('https://en.wikipedia.org/wiki/Midwifery', 3, 'midwifery'), ('https://en.wikipedia.org/wiki/Medical_education', 3, 'medical education'), ('https://en.wikipedia.org/wiki/National_Institutes_of_Health', 8, 'National Institutes of Health')]


In [60]:
contentParagraphsDF['source'] = [wikipedia_women_in_medicine] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,"Historically and presently, in many parts of t...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
1,1,,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
2,2,,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
3,3,"In 1540, Henry VIII of England granted the cha...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
4,4,At the beginning of the twenty-first century i...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
5,5,"However, the practice of medicine remains disp...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
6,6,"Moreover, there are skews within the medical p...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
7,7,Women continue to dominate in nursing. In 2000...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
8,8,Biomedical research and academic medical profe...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
9,9,The glass ceiling is used as a metaphor to con...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None


In [61]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

In [62]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,"Historically and presently, in many parts of t...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
1,1,,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
2,2,,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
3,3,"In 1540, Henry VIII of England granted the cha...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
4,4,At the beginning of the twenty-first century i...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
5,5,"However, the practice of medicine remains disp...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
6,6,"Moreover, there are skews within the medical p...",https://en.wikipedia.org/wiki/Women_in_medicine,None,None
7,7,Women continue to dominate in nursing. In 2000...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
8,8,Biomedical research and academic medical profe...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None
9,9,The glass ceiling is used as a metaphor to con...,https://en.wikipedia.org/wiki/Women_in_medicine,None,None


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [68]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":2976,"posts-type":false,"posts":[{"id":"170019185456","url":"http:\/\/lolcats-lol-cat.tumblr.com\/post\/170019185456","url-with-slug":"http:\/\/lolcats-lol-cat.tumblr.com\/post\/170019185456\/still-disappointed","type":"photo","date-gmt":"2018-01-23 01:00:14 GMT","date":"Tue, 23 Jan 2018 02:00:14","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1516669214,"format":"html","reblog-key":"Yv4ellVf","slug":"still-disappointed","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"170019185456\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_170019185456\"><iframe id=\"like_iframe_170019185456\" src=\"http:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=fc298e85f978b8662a643fe0a6b8c638#name=lolcats-lol-cat&amp;post_id=170019185456&am

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [69]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [ ]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [70]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,date,id,photo-type,photo-url,tags
0,"Tue, 23 Jan 2018 02:00:14",170019185456,jpg,http://78.media.tumblr.com/61a8463f9c5c6f8b4b5...,"[cat, cats, lol, lolcat, lolcats]"
1,"Mon, 22 Jan 2018 22:00:45",170011464953,jpg,http://78.media.tumblr.com/e7d91de227ad16982f2...,"[cat, cats, lol, lolcat, lolcats]"
2,"Fri, 19 Jan 2018 08:00:06",169877975025,png,http://78.media.tumblr.com/92fb68b60d7a58bfcbe...,"[cat, cats, lol, lolcat, lolcats]"
3,"Wed, 17 Jan 2018 04:00:20",169795782754,jpg,http://78.media.tumblr.com/586f473424d56bdea5c...,"[cat, cats, lol, lolcat, lolcats]"
4,"Wed, 17 Jan 2018 02:00:23",169792091886,png,http://78.media.tumblr.com/4e9f2da9e0beab9654a...,"[cat, cats, lol, lolcat, lolcats]"
5,"Tue, 16 Jan 2018 22:00:27",169784471102,jpg,http://78.media.tumblr.com/6d00e59b01dc3c54684...,"[cat, cats, lol, lolcat, lolcats]"
6,"Tue, 16 Jan 2018 20:00:18",169780780641,jpg,http://78.media.tumblr.com/6eb6debc4b73c1ca5c2...,"[cat, cats, lol, lolcat, lolcats]"
7,"Tue, 16 Jan 2018 18:00:25",169777309491,jpg,http://78.media.tumblr.com/7e5dbabc2b6f510be4e...,"[cat, cats, lol, lolcat, lolcats]"
8,"Tue, 16 Jan 2018 16:00:15",169774104413,jpg,http://78.media.tumblr.com/e3fb1fe9c5fab08f096...,"[cat, cats, lol, lolcat, lolcats]"
9,"Tue, 16 Jan 2018 14:00:15",169771310499,jpg,http://78.media.tumblr.com/40a8036b0d89376c207...,"[cat, cats, lol, lolcat, lolcats]"


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [71]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [72]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. LEts tart by looking at the Shakespeare files in the `data` directory 

In [73]:
with open('../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

, and Train.]

PUCK
  If we shadows have offended,
  Think but this,--and all is mended,--
  That you have but slumber'd here
  While these visions did appear.
  And this weak and idle theme,
  No more yielding but a dream,
  Gentles, do not reprehend;
  If you pardon, we will mend.
  And, as I am an honest Puck,
  If we have unearned luck
  Now to 'scape the serpent's tongue,
  We will make amends ere long;
  Else the Puck a liar call:
  So, good night unto you all.
  Give me your hands, if we be friends,
  And Robin shall restore amends.

[Exit.]





End of Project Gutenberg Etext of A Midsummer Night's Dream by Shakespeare
PG has multiple editions of William Shakespeare's Complete Works



Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [74]:
targetDir = '../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path) as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [75]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
alls_well_that_ends_well.txt,"All's Well, that Ends Well\n\nActus primus. Sc..."
anthonie_and_cleopatra.txt,"The Tragedie of Anthonie, and Cleopatra\n\nAct..."
as_you_like_it.txt,AS YOU LIKE IT\n\nby William Shakespeare\n\n\n...
comedy_of_errors.txt,"DRAMATIS PERSONAE\n\nSOLINUS, Duke of Ephesus\..."
coriolanus.txt,THE TRAGEDY OF CORIOLANUS\n\nby William Shakes...
cymbeline.txt,The Tragedie of Cymbeline\n\nActus Primus. Sco...
hamlet.txt,The Tragedie of Hamlet\n\nActus Primus. Scoena...
julius_caesar.txt,"Dramatis Personae\n\n JULIUS CAESAR, Roman st..."
king_henry_4_p1.txt,The First Part of Henry the Fourth\n\nwith the...
king_henry_4_p2.txt,"KING HENRY IV, SECOND PART\n\nby William Shake..."


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [76]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�ޚj9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&ү�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45�0�%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?�!IYi���U�"��y;���#��e3)�+B�&����<E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [77]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [78]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [79]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [80]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud` so we can use it later without having to retype it.

In [81]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [82]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [83]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [24]:
FAFRPOS_pdf = 'https://github.com/miriammidoun/Content-Analysis/blob/master/1-Intro/FA_FR_Pos_pdf.pdf'

In [25]:
FAFRPOSRequest = requests.get(FAFRPOS_pdf, stream=True)
print(FAFRPOSRequest.text[:1000])







<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://assets-cdn.github.com">
  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">



  <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/frameworks-98cac35b43fab8341490a2623fdaa7b696bbaea87bccf8f485fd5cdb4996cd9b52bdb24709fb3bab0a0dcff4a29187d65028ee693d609ce5c0c3283c77a247a9.css" media="all" rel="stylesheet" />
  <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/github-cabf8927ab4e896c0d65da4a3766408701184406f218800ef72483e3ad7369f3eaaad3d75e8e

In [26]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

In [27]:
FAFRPOSRequest = requests.get(FAFRPOS_pdf, stream=True)
print(FAFRPOSRequest.text[:1000])







<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://assets-cdn.github.com">
  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">



  <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/frameworks-98cac35b43fab8341490a2623fdaa7b696bbaea87bccf8f485fd5cdb4996cd9b52bdb24709fb3bab0a0dcff4a29187d65028ee693d609ce5c0c3283c77a247a9.css" media="all" rel="stylesheet" />
  <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/github-cabf8927ab4e896c0d65da4a3766408701184406f218800ef72483e3ad7369f3eaaad3d75e8e

In [28]:
requests.get(FAFRPOS_pdf)

<Response [200]>

In [30]:
FAFRPOSRequest = requests.get(FAFRPOS_pdf)
print(FAFRPOSRequest.text[:10000])







<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://assets-cdn.github.com">
  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">



  <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/frameworks-98cac35b43fab8341490a2623fdaa7b696bbaea87bccf8f485fd5cdb4996cd9b52bdb24709fb3bab0a0dcff4a29187d65028ee693d609ce5c0c3283c77a247a9.css" media="all" rel="stylesheet" />
  <link crossorigin="anonymous" href="https://assets-cdn.github.com/assets/github-cabf8927ab4e896c0d65da4a3766408701184406f218800ef72483e3ad7369f3eaaad3d75e8e